**Importing Libraries**

In [0]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from tqdm import tqdm

**Reading Train and Test probability values (generated by Model training code)***

In [0]:
df = pd.read_csv('pred_valuest.csv')
df.head()

In [0]:
df2 = pd.read_csv('pred_values_test.csv')
df2.head()

**Fusion Layer : Simple Exponential Smoothing**

**Finding Optimal parameter "alpha" through a grid search on training data**

In [0]:
opt_alpha_acc = 0
max_acc = -100
opt_alpha_loss = 0
min_loss = 100000
opt_alpha_f1 = 0
f1_max = -100

intervals = np.linspace(0.0, 1.0, num=21)
y = df.values[:,-1]

for alpha in tqdm(intervals):
    count = 0
    loss = 0
    preds = []
    for i in range(len(df)):
        data = df.values[i][:-1]
        model = SimpleExpSmoothing(data)
        model_fit = model.fit(alpha)
        prob = model_fit.forecast(1)[0]
        pred = np.round(prob)
        if pred==y[i]:
            count = count+1
        preds.append(pred)       
    acc = (count/len(df))*100.0
    loss = log_loss(y, preds)
    f1 =  f1_score(y, preds)
    if acc>max_acc:
        max_acc = acc
        opt_alpha_acc = alpha
        
    if loss<min_loss:
        min_loss = loss
        opt_alpha_loss = alpha
        
    if f1>f1_max:
        f1_max = f1
        opt_alpha_f1 = alpha

In [0]:
print(opt_alpha_acc)
print(max_acc)
print(opt_alpha_loss)
print(min_loss)
print(opt_alpha_f1)
print(f1_max)

**Making predictions on Test data**

In [0]:
from tqdm import tqdm
predictions = []
for i in tqdm(range(len(df2))):
        data = df2.values[i][1:]
        model = SimpleExpSmoothing(data)
        model_fit = model.fit(opt_alpha_f1)
        prob = model_fit.forecast(1)[0]
        pred = np.round(prob)
        if pred==1:
            predictions.append('SARCASM')
        else:
            predictions.append('NOT_SARCASM')

In [0]:
file1 = open("answer.txt","w")  

for i in range(len(predictions)):
  file1.write(df2['id'][i]+','+predictions[i]+'\n')

file1.close()

**Fusion Layer : Logistic Regression**

In [0]:
feature_cols = ['d1', 'd2', 'response']
X = df.loc[:, feature_cols]
y = df.y
X_new = df2.loc[:, feature_cols]

In [0]:
logreg = LogisticRegression()
logreg.fit(X, y)
new_pred_class = logreg.predict(X_new)

In [0]:
sar = []
for i in range(len(new_pred_class)):
  if(new_pred_class[i]==0.0):
    sar.append('NOT_SARCASM')
  else:
    sar.append('SARCASM') 

In [0]:
file1 = open("answer.txt","w")  

for i in range(len(sar)):
  file1.write(df2['id'][i]+','+sar[i]+'\n')

file1.close()